# Orders

✏️ **Exercise**

Today, we will investigate the **orders**, and their associated **review score**.

👉 Our goal is to create a DataFrame with the following features:


| feature_name 	| type 	| description 	|
|:---	|:---:	|:---	|
| `order_id` 	| str 	| the id of the order 	|
| `wait_time` 	| float 	| the number of days between order_purchase_timestamp and order_delivered_customer_date 	|
| `expected_wait_time` 	| float 	| the number of days between order_purchase_timestamp and estimated_delivery_date 	|
| `delay_vs_expected` 	| float 	| if the actual order_delivered_customer_date is later than the estimated delivery date, returns the number of days between the two dates, otherwise return 0 	|
| `order_status` 	| str 	| the status of the order 	|
| `dim_is_five_star` 	| int 	| 1 if the order received a five-star review, 0 otherwise 	|
| `dim_is_one_star` 	| int 	| 1 if the order received a one_star, 0 otherwise 	|
| `review_score` 	| int 	| from 1 to 5 	|
| `number_of_items` 	| int 	| number of items that the order contains 	|
| `number_of_sellers` 	| int 	| number of sellers involved in the order 	|
| `price` 	| float 	| total price of the order paid by customer 	|
| `freight_value` 	| float 	| value of the freight paid by customer 	|
| `distance_customer_seller` 	| float 	| the distance in km between customer and seller (optional) 	|  
  
⚠️ We also want to filter out "non-delivered" orders, unless explicitly specified, otherwise we cannot compute the potential delays.

❓ **Your challenge**: 

- Implement each feature as a separate method within the `Order` class available at `olist/order.py`
- Then, create a method `get_training_data()` that returns the complete DataFrame **without `NaN`s**.

💡 Suggested methodology:
- Use the notebook below to write and test your code step-by-step first
- Then copy the code into `order.py` once you are certain of your code logic
- Focus on the data manipulation logic now, we will analyse the dataset visually in the next challenges

🔥 Notebook best practices (must-read) 👇

<details>
    <summary>▸ <i>click here</i></summary>

From now on, exploratory notebooks are going to become pretty long, and we strongly advise you to follow these notebook principles:
- Code your logic so that your Notebook can always be ran from top to bottom without crashing (Run --> Run All Cells)
- Name your variables carefully 
- Use dummy names such as `tmp` or `_` for intermediary steps when you know you won't need them for long
- Clear your code and merge cells when relevant to minimize Notebook size (`Shift-M`)
- Make heavy use of of the collapsible headings and the table of content (call a TA if you can't find them)
- Use the following shortcuts 
    - `a` to insert a cell above
    - `b` to insert a cell below
    - `dd` to delete a cell
    - `esc` and `arrows` to move between cells
    - `Shift-Enter` to execute cell and move focus to the next one
    - use `Shift + Tab` when you are between method brackets e.g. `groupby()` to get the docs! Repeat a few times to open it permanently

</details>





In [1]:
# Auto reload imported module every time a jupyter cell is executed (handy for olist.order.py updates)
%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

In [4]:
# Import olist data
from olist.data import Olist
olist = Olist()
data = olist.get_data()

Starting get_data...
CSV path: /home/dunia-121/.lewagon/olist/data/csv
Found 9 files
File names: ['olist_sellers_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_order_items_dataset.csv', 'olist_customers_dataset.csv', 'olist_orders_dataset.csv', 'olist_order_payments_dataset.csv', 'product_category_name_translation.csv', 'olist_products_dataset.csv', 'olist_geolocation_dataset.csv']
Key names: ['sellers', 'order_reviews', 'order_items', 'customers', 'orders', 'order_payments', 'product_category_name_translation', 'products', 'geolocation']
Loaded 9 dataframes
Created dict with keys: ['sellers', 'order_reviews', 'order_items', 'customers', 'orders', 'order_payments', 'product_category_name_translation', 'products', 'geolocation']


In [5]:
# What datasets do we have access to now ?
data.keys()

dict_keys(['sellers', 'order_reviews', 'order_items', 'customers', 'orders', 'order_payments', 'product_category_name_translation', 'products', 'geolocation'])

In [7]:
orders = data['orders'].copy() # good practice to be sure not to modify your `data` variable

assert(orders.shape == (99441, 8))

## 1. Code `order.py`

### a) `get_wait_time`
    ❓ Return a Dataframe with:
           order_id, wait_time, expected_wait_time, delay_vs_expected, order_status


🎁 We give you the pseudo-code below 👇 for this first method:

> 1. Inspect the `orders` dataframe
2. Filter the dataframe on `delivered orders`
3. Handle `datetime`
    - Take time to understand what python [`datetime`](https://docs.python.org/3/library/datetime.html) objects are
    - and convert dates from "string" type to "pandas.datetime' using [`pandas.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)
4. Compute `wait_time` in days as a decimal number (starting from `order_purchase_timestamp`)
5. Compute `expected_wait_time` in days as a decimal number (starting from `order_purchase_timestamp`)
6. Compute `delay_vs_expected` in days as a decimal number (put `0` if the order is delivered earlier than estimated)
7. Check the new dataframe 
8. Once you are satisfied with your code, you can carefully copy-paste it from the notebook to to `olist/order.py`
9. Now try to load the data using your method in `orders.py`
10. Your code will probably not work straightaway
11. Make the necessary changes to make it work from inside a `.py` file

We want `wait_time`, `expected_wait_time` and `delay_vs_expected` in decimal numbers, not as integers. Think about it: a wait time of 8.1 or 8.9 days is a big difference. So you can't use Pandas' `dt.day` because that rounds both down to 8.

<details>
    <summary>💡Hint</summary>

For both `wait_time` and `delay_vs_expected`, you need to subtract the relevant dates/timestamps to get the time difference between the `pandas.datetime` objects. Then, you can either use [`datetime.timedelta()`](https://docs.python.org/3/library/datetime.html#timedelta-objects) or [`np.timedelta64()`](https://numpy.org/doc/stable/reference/arrays.datetime.html#datetime-and-timedelta-arithmetic) to find out how many days that subtraction represents!

</details>

In [ ]:

delivered_orders = orders[orders['order_status'] == 'delivered'].copy()


delivered_orders['order_purchase_timestamp'] = pd.to_datetime(delivered_orders['order_purchase_timestamp'])
delivered_orders['order_delivered_customer_date'] = pd.to_datetime(delivered_orders['order_delivered_customer_date'])
delivered_orders['order_estimated_delivery_date'] = pd.to_datetime(delivered_orders['order_estimated_delivery_date'])


delivered_orders['wait_time'] = (
    (delivered_orders['order_delivered_customer_date'] - delivered_orders['order_purchase_timestamp'])
    .dt.total_seconds() / 86400
)


delivered_orders['expected_wait_time'] = (
    (delivered_orders['order_estimated_delivery_date'] - delivered_orders['order_purchase_timestamp'])
    .dt.total_seconds() / 86400
)


delivered_orders['delay_vs_expected'] = (
    (delivered_orders['order_delivered_customer_date'] - delivered_orders['order_estimated_delivery_date'])
    .dt.total_seconds() / 86400
)
delivered_orders['delay_vs_expected'] = delivered_orders['delay_vs_expected'].apply(lambda x: x if x > 0 else 0)


wait_time_df = delivered_orders[[
    'order_id', 'wait_time', 'expected_wait_time', 'delay_vs_expected', 'order_status'
]]

wait_time_df.head()


,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,15.544063,0.0,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,19.137766,0.0,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,26.639711,0.0,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,26.188819,0.0,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,12.112049,0.0,delivered


👀 Check the dataframe you've just created. <br/> 

💪 When your code works, bring your code over to `olist/order.py` <br/>

🧪 Now, test it by running the following cell 👇 

In [21]:
# Test your code here
from olist.order import Order
Order().get_wait_time()


Starting get_data...
CSV path: /home/dunia-121/.lewagon/olist/data/csv
Found 9 files
File names: ['olist_sellers_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_order_items_dataset.csv', 'olist_customers_dataset.csv', 'olist_orders_dataset.csv', 'olist_order_payments_dataset.csv', 'product_category_name_translation.csv', 'olist_products_dataset.csv', 'olist_geolocation_dataset.csv']
Key names: ['sellers', 'order_reviews', 'order_items', 'customers', 'orders', 'order_payments', 'product_category_name_translation', 'products', 'geolocation']
Loaded 9 dataframes
Created dict with keys: ['sellers', 'order_reviews', 'order_items', 'customers', 'orders', 'order_payments', 'product_category_name_translation', 'products', 'geolocation']
Starting get_data...
CSV path: /home/dunia-121/.lewagon/olist/data/csv
Found 9 files
File names: ['olist_sellers_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_order_items_dataset.csv', 'olist_customers_dataset.csv', 'olist_orders_dataset.csv',

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,15.544063,0.0,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,19.137766,0.0,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,26.639711,0.0,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,26.188819,0.0,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,12.112049,0.0,delivered
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8.218009,18.587442,0.0,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,22.193727,23.459051,0.0,delivered
99438,83c1379a015df1e13d02aae0204711ab,24.859421,30.384225,0.0,delivered
99439,11c177c8e97725db2631073c19f07b62,17.086424,37.105243,0.0,delivered


In [22]:
from nbresult import ChallengeResult
test = Order().get_wait_time()
result = ChallengeResult('wait_time', dve_type=test["delay_vs_expected"].dtype, shape=test.shape, dve_min=test["delay_vs_expected"].min(), dve_max=test["delay_vs_expected"].max())
result.write(); 
print(result.check())

Starting get_data...
CSV path: /home/dunia-121/.lewagon/olist/data/csv
Found 9 files
File names: ['olist_sellers_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_order_items_dataset.csv', 'olist_customers_dataset.csv', 'olist_orders_dataset.csv', 'olist_order_payments_dataset.csv', 'product_category_name_translation.csv', 'olist_products_dataset.csv', 'olist_geolocation_dataset.csv']
Key names: ['sellers', 'order_reviews', 'order_items', 'customers', 'orders', 'order_payments', 'product_category_name_translation', 'products', 'geolocation']
Loaded 9 dataframes
Created dict with keys: ['sellers', 'order_reviews', 'order_items', 'customers', 'orders', 'order_payments', 'product_category_name_translation', 'products', 'geolocation']
Starting get_data...
CSV path: /home/dunia-121/.lewagon/olist/data/csv
Found 9 files
File names: ['olist_sellers_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_order_items_dataset.csv', 'olist_customers_dataset.csv', 'olist_orders_dataset.csv',

⚠️ Don't forget to also commit the code changes in `order.py` in your `olist` repo!

### b) `get_review_score`
     ❓ Returns a DataFrame with:
        order_id, dim_is_five_star, dim_is_one_star, review_score

dim_is_$N$_star should contain `1` if review_score=$N$ and `0` otherwise 

<details>
    <summary markdown='span'>Hints</summary>

Think about `Series.map()` or `DataFrame.apply()`
    
</details>

👉 We load the `reviews` for you

In [ ]:
reviews = data['order_reviews'].copy()
assert(reviews.shape == (99224,7))
reviews

In [ ]:
# YOUR CODE HERE

Once again, 

👀 Check the dataframe you've just created. <br/> 

💪 When your code works, bring your code over to `olist/order.py` <br/>

🧪 Now, test it by running the following cell 👇 

In [ ]:
# Test your code here
from olist.order import Order
Order().get_review_score()

In [ ]:
from nbresult import ChallengeResult
result = ChallengeResult('review_score', shape=Order().get_review_score().shape)
result.write(); print(result.check())

⚠️ Don't forget to also commit the code changes in `order.py` in your `olist` repo!

### c) `get_number_items`:
     ❓ Returns a DataFrame with:
        order_id, number_of_items (total number of items per order)

In [ ]:

data= olist.get_data()
data['order_items'].groupby('order_id').size().value_counts()

Starting get_data...
CSV path: /home/dunia-121/.lewagon/olist/data/csv
Found 9 files
File names: ['olist_sellers_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_order_items_dataset.csv', 'olist_customers_dataset.csv', 'olist_orders_dataset.csv', 'olist_order_payments_dataset.csv', 'product_category_name_translation.csv', 'olist_products_dataset.csv', 'olist_geolocation_dataset.csv']
Key names: ['sellers', 'order_reviews', 'order_items', 'customers', 'orders', 'order_payments', 'product_category_name_translation', 'products', 'geolocation']
Loaded 9 dataframes
Created dict with keys: ['sellers', 'order_reviews', 'order_items', 'customers', 'orders', 'order_payments', 'product_category_name_translation', 'products', 'geolocation']


order_id
00010242fe8c5a6d1ba2dd792cb16214    1
00018f77f2f0320c557190d7a144bdd3    1
000229ec398224ef6ca0657da4fc703e    1
00024acbcdf0a6daa1e931b038114c75    1
00042b26cf59d7ce69dfabb4e55b4fd9    1
                                   ..
fffc94f6ce00a00581880bf54a75a037    1
fffcd46ef2263f404302a634eb57f7eb    1
fffce4705a9662cd70adb13d4a31832d    1
fffe18544ffabc95dfada21779c9644f    1
fffe41c64501cc87c801fd61db3f6244    1
Length: 98666, dtype: int64

🧪 Same routine: 
* check your dataframe, 
* bring your code over to `olist/order.py`
* and check that it truly works.

In [ ]:
from nbresult import ChallengeResult
result = ChallengeResult('number_items',
                         shape=Order().get_number_items().shape,
                         columns=Order().get_number_items().columns)
result.write(); print(result.check())

⚠️ Don't forget to also commit the code changes in `order.py` in your `olist` repo!

### d) `get_number_sellers`:
     ❓ Returns a DataFrame with:
        order_id, number_of_sellers (total number of unique sellers per order)
        
<details>
    <summary>▸ <i>Hint</i></summary>

`pd.Series.nunique()`
</details>

In [ ]:
# YOUR CODE HERE

In [ ]:
from nbresult import ChallengeResult
result = ChallengeResult('number_sellers',
                         shape=Order().get_number_sellers().shape,
                         columns=Order().get_number_sellers().columns)
result.write(); print(result.check())

⚠️ Don't forget to also commit the code changes in `order.py` in your `olist` repo!

### e) `get_price_and_freight`
     Returns a DataFrame with:
        order_id, price, freight_value

In [ ]:
# YOUR CODE HERE

In [ ]:
from nbresult import ChallengeResult
result = ChallengeResult('price', shape=Order().get_price_and_freight().shape)
result.write(); print(result.check())

⚠️ Don't forget to also commit the code changes in `order.py` in your `olist` repo!

## 2. All at once: `get_training_data`

❓ Time to code `get_training_data` making use of your previous coded methods, to gather all order features in one table

In [ ]:
# YOUR CODE HERE

🧪  Test it below

In [ ]:
from nbresult import ChallengeResult
from olist.order import Order
data = Order().get_training_data()

result = ChallengeResult('training',
    shape=data.shape,
    columns=sorted(list(data.columns))
)
result.write()
print(result.check())

💡 **Not getting the exact number of rows?**
<details><summary>Do you have a difference of 8 rows?</summary>
At the end we want a Dataframe without NaNs. Try to get rid of those. (Don't make it too complex: you only need to change one line in your code.)
</details>
<details><summary>Do you have another difference?</summary>
Maybe something went wrong when merging all the steps in one Dataframe? Check your merges.
</details>

🏁 Congratulations! 

💾 Commit and push your notebook and your code in `olist/order.py` before starting the next challenge.

## 3. [OPTIONAL] `get_distance_seller_customer` 
**Try to code this function only after finishing today's challenges.**

**First skip to the next challenge.**

    ❓ Returns a Dataframe with:
        order_id, distance_seller_customer (the distance in km between customer and seller)

💡 Have a look at the `haversine_distance` formula we coded for you in the `olist.utils` module

Use this to code `get_distance_seller_customer`.

⚠️ Since an order can have multiple sellers, take the average seller-customer distance per order.

In [ ]:
# YOUR CODE HERE

In [ ]:
matching_geo = Order().get_distance_seller_customer()

👉 Let's quickly check the distribution of the distance seller-customer :)

In [ ]:
sns.displot(data = matching_geo, x ='distance_seller_customer')

👉 What is the median and the average seller-customer distance ?

In [ ]:
matching_geo['distance_seller_customer'].describe()

👀 Check your new dataframe and bring your code over to olist/order.py when it works. 

In [ ]:
Order().get_distance_seller_customer()['distance_seller_customer']

🧪  Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('distance',
    mean = Order().get_distance_seller_customer()['distance_seller_customer'].mean())
result.write()
print(result.check())

Now also update `get_training_data` to merge the `get_distance_seller_customer` if `with_distance_seller_customer` is set to `True`.

⚠️ Don't forget to also commit the code changes in `order.py` in your `olist` repo!